In [25]:
import pandas as pd
import numpy as np
import replicaEVSE.load_curve as sim
import replicaEVSE.datautils as simdu
import os
import joblib
import dask.dataframe as dd
from tqdm import tqdm

%reload_ext autoreload
%autoreload 2

pd.set_option('display.max_columns', None)

datadir = '../../data/'
mode = 'PRIVATE_AUTO'
test = False

# Sample the number of people/vehicles by county segment and year

see  `scripts/sample_counties.py` to run 

In [2]:


#Created in the EIA_data_download.ipynb notebook
existing_load=pd.read_csv(datadir+'EIA_demand_summary.csv') 
if test:
    # df = pd.read_parquet(os.path.join(datadir, 'wa_pop_and_trips_subsample.parquet'))
    df = pd.read_parquet(os.path.join(datadir, 'wa_pop_and_trips_sorted_county.parquet'))
    df = df.head(10000)
    df = df.loc[df['mode'] == mode]
    simulation_id = 'dev'

else: 
    # read in the joined trips and population data sets
    merged_df = pd.read_parquet(os.path.join(datadir, 'wa_pop_and_trips_sorted_county.parquet'))

    # right now, only look at private auto trips
    df = merged_df.loc[merged_df['mode'] == mode]
    # take out the mobile and commercial MHDV

### TODO: revisit taking out mobile home owners
df = df[(df['building_type'] != 'mobile') & (df['building_type'] != None)]



###  sample by county: for now use a percent until we have the stock rollover model

`ev_df` should be given to us to run. 

In [3]:
stock_rollover = pd.read_csv(datadir+'ldv_population_output_adjusted.csv')
efficiency = pd.read_csv(datadir+'vehicle_inputs.csv')

In [4]:
personal = ['Personal Sedan', 'Personal Crossover', 'Personal Truck/SUV']
commercial = ['Commercial Sedan', 'Commercial Crossover', 'Commercial Truck/SUV']
for cartype in personal + commercial:
    for powertrain in  ['EV', 'PHEV']:
        cond = (efficiency['Vehicle_type']==cartype) & ~efficiency['Powertrain'].isin(['ICE-G', 'ICE-D', 'FCEV']) & (efficiency['Powertrain']==powertrain) & (efficiency['Vintage'] == 2022)
        ef = efficiency[cond]['Efficiency'].values[0]
        print(cartype, powertrain, ef)

Personal Sedan EV 0.25
Personal Sedan PHEV 80.0
Personal Crossover EV 0.3
Personal Crossover PHEV 80.0
Personal Truck/SUV EV 0.49
Personal Truck/SUV PHEV 80.0
Commercial Sedan EV 0.25
Commercial Sedan PHEV 80.0
Commercial Crossover EV 0.3
Commercial Crossover PHEV 80.0
Commercial Truck/SUV EV 0.49
Commercial Truck/SUV PHEV 80.0


# extract the population ldv data from the stock rollover model 
 - What to do about housing == 'other'? 

### include both EV and PHEV

In [5]:
ev_cond = stock_rollover['Powertrain'].isin(['EV', 'PHEV']).reset_index(drop=True)
# ev_cond = (stock_rollover['Powertrain']=='EV') | (stock_rollover['Powertrain']=='PHEV')
# nev_df = stock_rollover[(stock_rollover['Powertrain']=='EV')].copy()
nev_df = stock_rollover[ev_cond].copy()
nev_df = nev_df[nev_df['domicile'] != 'other'].copy()
nev_df.drop(columns=['Unnamed: 0'], inplace=True)
print(len(nev_df))


924


See the change in stock over the years

In [6]:
nev_df[['2022', '2035']]

,2022,2035
0,0,1
3,0,0
4,0,5
7,0,1
8,0,0
...,...,...
1839,98,14859
1840,34,22314
1843,18,5020
1844,133,83107


We ingest the number of vehicles in each segment for each year and sample from that subset of the trips table for each county segment and year. This takes > 300 minutes for all the years and segments

In [7]:
nev_df

,County,Vehicle_type,domicile,Powertrain,2022,2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034,2035
0,Adams,Commercial Crossover,mfh,EV,0,0,0,0,0,0,0,1,1,1,1,1,1,1
3,Adams,Commercial Crossover,mfh,PHEV,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Adams,Commercial Crossover,sfh,EV,0,0,1,1,1,2,2,2,3,3,3,5,5,5
7,Adams,Commercial Crossover,sfh,PHEV,0,1,1,1,1,1,1,1,1,1,1,1,1,1
8,Adams,Commercial Sedan,mfh,EV,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1839,Yakima,Personal Sedan,sfh,PHEV,98,2433,4815,6453,7994,9117,10046,10703,11606,12252,12842,13552,14206,14859
1840,Yakima,Personal Truck/SUV,mfh,EV,34,1780,3504,5143,6768,8393,9963,11285,12851,14524,16279,18064,19987,22314
1843,Yakima,Personal Truck/SUV,mfh,PHEV,18,1063,1757,2265,2617,2945,3170,3367,3593,3845,4102,4380,4667,5020
1844,Yakima,Personal Truck/SUV,sfh,EV,133,6521,12813,18796,24713,30637,36335,41141,46932,53211,59811,66703,74132,83107


In [16]:
# get the unique people in the dataframe
pop_df = df.drop_duplicates(subset=['person_id'])[['person_id', 'destination_county', 'building_type']]

In [33]:
# test run on a single year ~14 mins
final_df = simdu.sample_people_by_county(df, nev_df, year=2025, fraction=None)

Selecting people from each county in year=2025...14 mins per year


In [34]:
final_df.head()[['person_id', 'distance_miles', 'destination_county', 'building_type', 'engine', 'segment', 'efficiency', 'year']]

,activity_id,person_id,mode,travel_purpose,previous_activity_type,start_time,end_time,distance_miles,vehicle_type,origin_bgrp,origin_bgrp_lat,origin_bgrp_lng,destination_bgrp,destination_bgrp_lat,destination_bgrp_lng,origin_land_use_l1,origin_land_use_l2,origin_building_use_l1,origin_building_use_l2,destination_land_use_l1,destination_land_use_l2,destination_building_use_l1,destination_building_use_l2,origin_lat,origin_lng,destination_lat,destination_lng,weekday,household_id,BLOCKGROUP,BLOCKGROUP_work,BLOCKGROUP_school,TRACT,TRACT_work,TRACT_school,age_group,age,sex,race,ethnicity,individual_income_group,individual_income,employment,education,school_grade_attending,industry,household_role,subfamily_number,subfamily_relationship,commute_mode,tenure,migration,household_size,household_income_group,household_income,family_structure,vehicles,building_type,resident_type,language,lat,lng,lat_work,lng_work,lat_school,lng_school,wfh,charge_type,destination_county,engine,segment,efficiency,year
23982863,16445481232436098279,3105704646168074816,PRIVATE_AUTO,RECREATION,SOCIAL,0 days 12:15:50,0 days 12:56:28,4.782405,nan,530019503001,46.785525,-119.181545,530019504001,46.828923,-119.159610,residential,single_family,residential,single_family,commercial,non_retail_attraction,commercial,non_retail_attraction,46.790228,-119.215020,46.825792,-119.163902,thursday,7270493665717972005,530019503001,530019504002,nan,53001950300,53001950400,nan,35_64,55.0,M,white,not_hispanic_or_latino,40000_80000,43867.0,employed,some_college,not_attending_school,naics423450,spouse,0,nan,driving,owner,same_house,7_plus_person,40000_75000,67529.0,married_couple,3_plus,single_family,core,english,46.790228,-119.215020,46.825830,-119.174831,NaN,NaN,worked_from_home,PUBLIC,"Adams County, WA",EV,Commercial Crossover,0.30,2025
23982866,14816401451004063957,3105704646168074816,PRIVATE_AUTO,SHOP,RECREATION,0 days 16:33:49,0 days 16:34:35,0.176954,nan,530019504001,46.828923,-119.159610,530019504002,46.832235,-119.171912,commercial,non_retail_attraction,commercial,non_retail_attraction,commercial,retail,commercial,retail,46.825792,-119.163902,46.826474,-119.166795,thursday,7270493665717972005,530019503001,530019504002,nan,53001950300,53001950400,nan,35_64,55.0,M,white,not_hispanic_or_latino,40000_80000,43867.0,employed,some_college,not_attending_school,naics423450,spouse,0,nan,driving,owner,same_house,7_plus_person,40000_75000,67529.0,married_couple,3_plus,single_family,core,english,46.790228,-119.215020,46.825830,-119.174831,NaN,NaN,worked_from_home,PUBLIC,"Adams County, WA",EV,Commercial Crossover,0.30,2025
23982867,17546959975451274985,3105704646168074816,PRIVATE_AUTO,SHOP,SHOP,0 days 16:48:41,0 days 17:21:53,26.754191,nan,530019504002,46.832235,-119.171912,530250111003,47.117839,-119.303234,commercial,retail,commercial,retail,commercial,retail,commercial,retail,46.826474,-119.166795,47.124822,-119.290904,thursday,7270493665717972005,530019503001,530019504002,nan,53001950300,53001950400,nan,35_64,55.0,M,white,not_hispanic_or_latino,40000_80000,43867.0,employed,some_college,not_attending_school,naics423450,spouse,0,nan,driving,owner,same_house,7_plus_person,40000_75000,67529.0,married_couple,3_plus,single_family,core,english,46.790228,-119.215020,46.825830,-119.174831,NaN,NaN,worked_from_home,PUBLIC,"Grant County, WA",EV,Commercial Crossover,0.30,2025
23982868,2138951794206724443,3105704646168074816,PRIVATE_AUTO,HOME,SHOP,0 days 17:28:52,0 days 20:13:16,32.847668,nan,530250111003,47.117839,-119.303234,530019503001,46.785525,-119.181545,commercial,retail,commercial,retail,residential,single_family,residential,single_family,47.124822,-119.290904,46.790228,-119.215020,thursday,7270493665717972005,530019503001,530019504002,nan,53001950300,53001950400,nan,35_64,55.0,M,white,not_hispanic_or_latino,40000_80000,43867.0,employed,some_college,not_attending_school,naics423450,spouse,0,nan,driving,owner,same_house,7_plus_person,40000_75000,67529.0,married_couple,3_plus,single_family,cor

### Can parallelize this by looping over years

In [36]:
year_list = np.arange(2022, 2036, 1)
joblib.Parallel(verbose=10, n_jobs=len(year_list))(joblib.delayed(simdu.run_and_save_sampled_populations)(
    df,
    nev_df, 
    year,
    datadir,
    ) for year in year_list)


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 64 concurrent workers.


Selecting people from each county in year=2022...14 mins per year
Selecting people from each county in year=2023...14 mins per year
Selecting people from each county in year=2024...14 mins per year
Selecting people from each county in year=2025...14 mins per year
Selecting people from each county in year=2026...14 mins per year
Selecting people from each county in year=2027...14 mins per year


/home/matthew.wilde@RMI.local/mambaforge/envs/py311/lib/python3.11/site-packages/joblib/externals/loky/process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Selecting people from each county in year=2028...14 mins per year
Selecting people from each county in year=2029...14 mins per year
Selecting people from each county in year=2030...14 mins per year
Selecting people from each county in year=2031...14 mins per year


: 

: 